In [1]:
import math
import pandas as pd
import numpy as np
from scipy import optimize 
from sklearn.neural_network import MLPRegressor

In [2]:
data = pd.read_csv('workouts.csv')
data.describe()
Actual_Performance = data['IF']
Offset_Performance = []
for i in range(len(Actual_Performance)):
    Offset_Performance.append(np.mean(Actual_Performance[i: (i+28)])) 
TSS = data['TSS']
Block_TSS = [] 
for i in range(len(TSS)):
    avg_TSS = np.mean(TSS[i:(i+28)])
    Block_TSS.append(avg_TSS)


In [3]:
def calc_vo2_if_bike(row, max_hr, resting_hr, weight) :
    if row['WorkoutType'] == 'Bike':
        percent_vo2 = (row['HeartRateAverage'] - resting_hr) / (max_hr - resting_hr)
        vo2_power = row['PowerAverage'] / percent_vo2
        vo2_estimated = (((vo2_power)/75)*1000)/weight
        return vo2_estimated


In [4]:
def optimize_banister(params):
    data = pd.read_csv('workouts2.csv')
    data['day_TSS'] = data['TSS'].groupby(data['WorkoutDay']).transform('sum') #Fill in any missing days with zero
    data['day_TSS'] = data['day_TSS'].fillna(0) #Fill in any missing days with zero
    data['bike_V02'] = data.apply(lambda row: calc_vo2_if_bike(row, 196, 50, 74), axis=1) #Calculate VO2 for each bike workout
    data = data[['WorkoutDay', 'day_TSS', 'bike_V02']] #Keep only the columns we need
    data = data.groupby('WorkoutDay').mean() #Group by day and take the mean of the TSS and VO2
    data['WorkoutDate'] = data.index #Add a column for the date
    data['WorkoutDate'] = pd.to_datetime(data['WorkoutDate']) #Convert the date column to a datetime object
    data = data.sort_values(by=['WorkoutDate']) #Sort the data by date
    data.index = pd.DatetimeIndex(data['WorkoutDate']) #Set the index to the date
    missing_dates = pd.date_range(start=data.index.min(), end=data.index.max()) #Create a list of all the dates in the range
    data = data.reindex(missing_dates, fill_value=0) #Add missing dates
    data['bike_V02'] = data['bike_V02'].fillna(method="ffill") #FiLL missing VOZ values with previous value
    data = data.dropna() #Drop any remaining rows with missing values TSS= data['day_TSS'].to_list() #Convert the TSS column to a list
    TSS = data['day_TSS'].to_list() #Convert the TSS column to a list
    Performance = data['bike_V02'].to_list() #Convert the V02 column to a list
    losses = [] #Create an empty list to store the loss values from our model
    ctls = [63] #Create an empty list to store the CTL values from our model with starting CTL of 0
    atls = [63] #Create an empty list to store the ATL values from our model with starting ATL of 0 for i in range(len(TSS)):
    tsbs = [0]
    for i in range(len(TSS)):
        ctl = (TSS[i] * (1-math.exp(-1/params[3]))) + (ctls[i] * (math.exp(-1/params[3]))) #Calculate the CTL for the day
        atl = (TSS[i] * (1-math.exp(-1/params[4]))) + (atls[i] * (math.exp(-1/params[4]))) #Calculate the ATL for the day
        tsb = ctl - atl #Calculate the TSB for the day
        ctls.append(ctl) #Add the CTL to the list
        atls.append(atl) #Add the ATL to the List
        tsbs.append(tsb) #Add the TSB to the list
        Banister_Prediction = params[2] + params[0]*ctl + params[1]*tsb #Calculate the Banister Prediction for the day
        loss = (Performance[i]- Banister_Prediction)**2
        losses.append(loss) #Add the loss to the list
    # print(f"CTLs: {ctls}")
    # print(f"ATLS: {atls}")
    MAE= np.mean(losses) #Calculate the mean absolute error
    RMSE = np.nanmean(losses)**0.5
    # print(f"MAE: {MAE}")
    print(f"k1: {params[0]} k2: {params[1]} Tau1: {params[3]} Tau2: {params[4]} PO: {params[2]} RMSE: {RMSE}")
    return MAE

In [5]:
# k1, k2, PO, Tau1, Tau2
initial_guess = [2, 2, 63, 45, 15]
individual_banister_model = optimize.minimize(optimize_banister,  x0 = initial_guess, bounds=[(0,5),(0,5),(30,100),(20,80),(10,20)])
# print(individual_banister_model['x'])

# Individual Neural Network Model
individual_neural_net_model = MLPRegressor(solver='lbfgs', activation='relu', hidden_layer_sizes=[50], random_state=42)
print(Block_TSS)
print(Offset_Performance)

k1: 2.0 k2: 2.0 Tau1: 45.0 Tau2: 15.0 PO: 63.0 RMSE: 139.1897832331658
k1: 2.00000001 k2: 2.0 Tau1: 45.0 Tau2: 15.0 PO: 63.0 RMSE: 139.18978395354412
k1: 2.0 k2: 2.00000001 Tau1: 45.0 Tau2: 15.0 PO: 63.0 RMSE: 139.18978317934034
k1: 2.0 k2: 2.0 Tau1: 45.0 Tau2: 15.0 PO: 63.00000001 RMSE: 139.18978324308884
k1: 2.0 k2: 2.0 Tau1: 45.00000001 Tau2: 15.0 PO: 63.0 RMSE: 139.18978322819686
k1: 2.0 k2: 2.0 Tau1: 45.0 Tau2: 15.00000001 PO: 63.0 RMSE: 139.18978323801724
k1: 0.0 k2: 5.0 Tau1: 80.0 Tau2: 10.0 PO: 30.0 RMSE: 91.11882402141279
k1: 1e-08 k2: 5.0 Tau1: 80.0 Tau2: 10.0 PO: 30.0 RMSE: 91.11882342929441
k1: 0.0 k2: 4.99999999 Tau1: 80.0 Tau2: 10.0 PO: 30.0 RMSE: 91.11882388452266
k1: 0.0 k2: 5.0 Tau1: 80.0 Tau2: 10.0 PO: 30.00000001 RMSE: 91.11882401327914
k1: 0.0 k2: 5.0 Tau1: 79.99999999 Tau2: 10.0 PO: 30.0 RMSE: 91.11882401761599
k1: 0.0 k2: 5.0 Tau1: 80.0 Tau2: 10.00000001 PO: 30.0 RMSE: 91.1188239971738
k1: 0.72971921877933 k2: 3.60383444740195 Tau1: 64.7969617378051 Tau2: 12.19369

A larger k1 than k2 indicates an individual takes longer to recover, whereas a larger k2 than k1 indicates a faster recovery.

k1, k2, PO, Tau1, Tau2

Pawel:


Tomek:
[ 0.          0.21154465 58.90335039 60.         20.        ]


In [6]:
arr = np.array(Block_TSS)
arr_Block_TSS = arr.reshape(-1,1)
# print(arr_Block_TSS)

individual_neural_net_model.fit(arr_Block_TSS, Offset_Performance)
# print(individual_neural_net_model.predict(arr_Block_TSS))
# individual_neural_net_model_score = individual_neural_net_model.score(arr_Block_TSS, Offset_Performance)
# individual_neural_net_model.fit([[]100],[0.5])




MLPRegressor(hidden_layer_sizes=[50], random_state=42, solver='lbfgs')